In [ ]:
import json
import subprocess
import os

import altair as alt
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# alt.data_transformers.enable('csv')
alt.data_transformers.disable_max_rows()

In [ ]:
ips = [
    "34.69.104.150",
    "34.171.40.202",
    "104.154.225.158",
    "35.222.168.210",
    "34.132.141.156",
]

In [ ]:
# for i, ip in tqdm(enumerate(ips)):
#     path = f'../outputs/ss-skip'
#     if not os.path.exists(path):
#         os.makedirs(path)
#     process = subprocess.Popen(f'scp -r -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null chanwutk@{ip}:"/home/chanwutk/Documents/apperception/outputs/run/*" {path}', shell=True)
#     process.wait()

In [ ]:
DATA_DIR = '../outputs/ss-skip'
runs = [f'/run-{i}' for i in range(len(ips))]

In [ ]:
failed_videos = {}
for run in runs:
    for f in os.listdir(DATA_DIR):
        if not f.startswith('failed_videos'):
            continue

        with open(os.path.join(DATA_DIR, f), 'r') as _f:
            content = json.load(_f)

        name = f[len('failed_videos--'):-len('.json')]
        failed_videos[name] = []
        for v_name, err in content:
            failed_videos[name].append(v_name)
failed_videos

In [ ]:
stage_order = [
    'InView', 'DecodeFrame', 'Detection2D.YoloDetection', 'Detection2D.ObjectTypeFilter',
    'DepthEstimation', 'Detection3D.FromDetection2DAndDepth', 'Detection3D.FromDetection2DAndRoad',
    'DetectionEstimation', 'Tracking2D.StrongSORT', 'Tracking3D.FromTracking2DAndDepth',
    'Tracking3D.FromTracking2DAndRoad', 'SegmentTrajectory.FromTracking3D'
]

test_names = {
    'de': 'Only Detection Estimation',
    'noopt': 'Baseline',
    'inview': 'Only In-View',
    'geo': 'Only Geo Depth Estimation',
    'objectfilter': 'Only Object Filter',
    'opt': 'Optimized',
    'optde': 'Optimized with Detection Estimation'
}

test_order = [
    'noopt',
    'inview',
    'objectfilter',
    'geo',
    'de',
    'opt',
    'optde'
]

In [ ]:
def combine_perf():
    data = []
    for filename in os.listdir(DATA_DIR):
        if filename.startswith('perf'):
            test, run = filename.split("--")[1].split(".")[0].split("_")
            with open(os.path.join(DATA_DIR, filename), 'r') as f:
                for stage in json.load(f):
                    stagename = stage['stage']
                    for b in stage['benchmark']:
                        after, before = b['keep']
                        assert after <= before
                        skip = (1. - (after / before)) if before != 0 else 0
                        data.append({
                            **b,
                            'stage': (stagename
                                      .replace('Detection2D', 'D2D')
                                      .replace('Detection3D', 'D3D')
                                      .replace('Tracking2D', 'T2D')
                                      .replace('Tracking3D', 'T3D')
                                      .replace('SegmentTrajectory', 'ST')),
                            '_test': test,
                            'test': test,
                            # 'test': test_names[test],
                            # 'test_order': test_order.index(test),
                            'run': run,
                            'name': b['name'].split('/')[-1].split('.')[0],
                            'skip': skip,
                            # 'stage_order': stage_order.index(stagename)
                        })
    return data


perf = combine_perf()

In [ ]:
# with open('./output/perf.json', 'w') as f:
#     json.dump([p for p in perf if p['run'] == "0"], f, indent=1)

In [ ]:
len([p for p in perf if p['run'] == "0"])

In [ ]:
perf[0]

In [ ]:
for name, videos in failed_videos.items():
    if len(videos) != 0:
        print(name, videos)

In [ ]:
df_perf = pd.DataFrame.from_dict(perf)
# df_perf = df_perf[df_perf['run'] == "0"]
df_perf[:10]

In [ ]:
df_perf.groupby(['test', 'stage'])[['name']].count()

In [ ]:
(alt.Chart(df_perf)
    .mark_bar()
    .encode(
        x='average(runtime)',
        y=alt.Y('test:O', sort=alt.Sort({'field': 'test_order'})),
        color=alt.Color('stage:N', sort=stage_order),
        order=alt.Order('order:O')
    )
    .properties(height=300, width=800)
)

In [ ]:
def combine_skip():
    data = []
    for dirname in os.listdir(DATA_DIR):
        if dirname.startswith('sort'):
            test, run = dirname.split("--")[1].split("_")
            for filename in os.listdir(os.path.join(DATA_DIR, dirname)):
                with open(os.path.join(DATA_DIR, dirname, filename), 'r') as f:
                    data.append({
                        'name': filename.split('.')[0],
                        'associations': json.load(f)
                    })
            # with open(os.path.join(DATA_DIR, dirname), 'r') as f:
            #     for stage in json.load(f):
            #         stagename = stage['stage']
            #         for b in stage['benchmark']:
            #             after, before = b['keep']
            #             assert after <= before
            #             skip = (1. - (after / before)) if before != 0 else 0
            #             data.append({
            #                 **b,
            #                 'stage': (stagename
            #                           .replace('Detection2D', 'D2D')
            #                           .replace('Detection3D', 'D3D')
            #                           .replace('Tracking2D', 'T2D')
            #                           .replace('Tracking3D', 'T3D')
            #                           .replace('SegmentTrajectory', 'ST')),
            #                 '_test': test,
            #                 'test': test,
            #                 # 'test': test_names[test],
            #                 # 'test_order': test_order.index(test),
            #                 'run': run,
            #                 'name': b['name'].split('/')[-1].split('.')[0],
            #                 'skip': skip,
            #                 # 'stage_order': stage_order.index(stagename)
            #             })
    return data


skip = combine_skip()

In [ ]:
# skip[0]

In [ ]:
for video in skip:
    name = video['name']
    associations = video['associations']
    for idx, frame in enumerate(associations):
        # print(len(frame), idx % 10 == 0)
        if idx % 10 == 0:
            # frames with skip testing
            assert len(frame) > 1 or idx == len(associations) - 1, len(frame)
            for idx_, frame_ in enumerate(frame):
                print(len(frame_), len(associations[idx + idx_][0]))
        else:
            # regular frame
            assert len(frame) == 1
    # break
# todo: rerun the benchmark -> need to run at least 2 frames

# Comparing the Effect of Detection Estimation

In [ ]:
df = df_perf[(df_perf.stage == 'DetectionEstimation') | (df_perf.stage == 'T2D.StrongSORT')]
df_noopt = df[df._test == 'noopt']
df_de = df[df._test == 'de']
df_opt = df[df._test == 'opt']
df_optde = df[df._test == 'optde']

In [ ]:
len(df_noopt), len(df_de[df_de.run == '0'])

In [ ]:
def join(baseline: "pd.DataFrame", de: "pd.DataFrame"):
    indices = ['name', 'run']

    def _(x):
        return pd.Series(
            [x.runtime_ss_baseline, x.keep_ss_baseline[1], x.runtime_de_de, x.runtime_ss_de, x.keep_de_de[0], x.run, x['name']],
            index=['runtime_before', 'frames_before', 'runtime_ss_after', 'runtime_de_after', 'frames_after', 'run', 'name']
        )

    return (
        baseline[baseline.stage == 'T2D.StrongSORT']
            .set_index(indices)
            .join(de[de.stage == 'T2D.StrongSORT'].set_index(indices), lsuffix="_noOpt", rsuffix="_de", how='inner')
            .join(de[de.stage == 'DetectionEstimation'].set_index(indices), lsuffix="_SS", rsuffix="_DE", how='inner')
            .rename(columns={
                'runtime_noOpt': 'runtime_ss_baseline',
                'keep_noOpt': 'keep_ss_baseline',
                'stage_noOpt': 'stage_ss_baseline',
                'runtime_de': 'runtime_ss_de',
                'keep_de': 'keep_ss_de',
                'stage_de': 'stage_ss_de',
                'runtime': 'runtime_de_de',
                'keep': 'keep_de_de',
                'stage': 'stage_de_de',
            })
            .reset_index()
            .apply(_, axis=1)
    )

In [ ]:
df_baseline = join(df_noopt, df_de)
df_baseline

In [ ]:
df_optimized = join(df_opt, df_optde)
df_optimized

In [ ]:
def scatter_runtime_frame(baseline, optimized):
    def one_chart(data, name):
        threshold = pd.DataFrame([{"threshold": 0}])
        # TODO: use y = x
        line = alt.Chart(threshold).mark_rule(color='black').encode(y='threshold:Q')
        base = (
            alt.Chart(data)
            .transform_calculate(
                'runtime_reduction',
                calculate='(datum.runtime_ss_after + datum.runtime_de_after) / datum.runtime_before - 1'
            )
            .transform_calculate(
                'frames_reduction',
                calculate='1 - datum.frames_after / datum.frames_before'
            )
            .transform_filter('datum.run === "0"')
            .encode(
                x=alt.X('frames_reduction:Q', title='Frame Skip Rate', scale=alt.Scale(domain=[0, 1])),
                y=alt.Y('runtime_reduction:Q', title='Runtime Reduction (+ is increase)', scale=alt.Scale(domain=[-1, 1])),
            )
        )
        chart = (
            base
            .mark_point()
            .properties(title=f"{name}: Runtime Reduction vs Frame Skip Rate")
        )

        return chart + line
    return alt.hconcat(one_chart(baseline, 'Baseline'), one_chart(optimized, 'Optimized'))


scatter_runtime_frame(df_baseline, df_optimized)

In [ ]:
def scatter_runtime_de(baseline, optimized):
    def one_chart(data, name):
        threshold = pd.DataFrame([{"threshold": 0}])
        line = alt.Chart(threshold).mark_rule(color='black').encode(y='threshold:Q')
        chart = (
            alt.Chart(
                data,
                title=f"{name}: Runtime Reduction vs Detection Estimation Runtime"
            )
            .transform_calculate(
                'runtime_reduction',
                calculate='1 - (datum.runtime_ss_after + datum.runtime_de_after) / datum.runtime_before'
            )
            .transform_filter('datum.run === "0"')
            .mark_point()
            .encode(
                x=alt.X('runtime_de_after:Q', title='Detection Estimation Runtime', scale=alt.Scale(domain=[0, 130])),
                y=alt.Y('runtime_reduction:Q', title='Runtime Reduction', scale=alt.Scale(domain=[-1, 1])),
            )
        ) 

        return chart + line
    return alt.hconcat(one_chart(baseline, 'Baseline'), one_chart(optimized, 'Optimized'))


scatter_runtime_de(df_baseline, df_optimized)

In [ ]:
def scatter_frame_de(baseline, optimized):
    def one_chart(data, name):
        threshold = pd.DataFrame([{"threshold": 0}])
        # TODO: use y = 1 - x
        # line = alt.Chart(threshold).mark_rule(color='red').encode(y='threshold:Q')
        chart = (
            alt.Chart(
                data,
                title=f"{name}: Frame Skip Rate vs Detection Estimation Runtime"
            )
            .mark_point()
            .transform_calculate(
                'frames_reduction',
                calculate='1 - datum.frames_after / datum.frames_before'
            )
            .transform_filter('datum.run === "0"')
            .encode(
                x=alt.X('runtime_de_after:Q', title='Detection Estimation Runtime', scale=alt.Scale(domain=[0, 130])),
                y=alt.Y('frames_reduction:Q', title='Frame Skip Rate', scale=alt.Scale(domain=[0, 1])),
            )
        )

        # return chart + line
        return chart
    return alt.hconcat(one_chart(baseline, 'Baseline'), one_chart(optimized, 'Optimized'))


scatter_frame_de(df_baseline, df_optimized)

In [ ]:
def cdf_skip(baseline, optimized):
    # def one_chart(data, name):
    return (
        alt.Chart(
            pd.concat([baseline.assign(test='Baseline'), optimized.assign(test='Optimized')]),
            title='CDF of Detection Estimation\'s skip ratio'
        )
        .transform_calculate(
            'skip_ratio',
            calculate='1 - (datum.frames_after / datum.frames_before)'
        )
        .transform_window(
            ECDF="cume_dist()",
            groupby=["test"],
            sort=[{"field": "skip_ratio"}],
        )
        .mark_line()
        .encode(
            x=alt.X("skip_ratio:Q", title='Frame Skip Ratio'),
            y="ECDF:Q",
            color="test:N"
        )
    )
    # return alt.hconcat(one_chart(baseline, 'Baseline'), one_chart(optimized, 'Optimized'))


cdf_skip(df_baseline, df_optimized)

In [ ]:
def cdf_runtime(baseline, optimized):
    # def one_chart(data, name):
    chart = (
        alt.Chart(
            pd.concat([baseline.assign(test='Baseline'), optimized.assign(test='Optimized')]),
            title='CDF of Runtime Reduction (%)'
        )
        .transform_calculate(
            'runtime_reduction',
            calculate='100*(1 - ((datum.runtime_ss_after + datum.runtime_de_after) / datum.runtime_before))'
        )
        .transform_window(
            ECDF="cume_dist()",
            groupby=['test'],
            sort=[{"field": "runtime_reduction"}],
        )
        .mark_line()
        .encode(
            x=alt.X("runtime_reduction:Q", title='Runtime Reduction (%)'),
            y="ECDF:Q",
            color='test:N'
        )
    )
    return alt.hconcat(chart, chart.transform_filter('datum.runtime_reduction > -100'))
    # return alt.hconcat(one_chart(baseline, 'Baseline'), one_chart(optimized, 'Optimized'))


cdf_runtime(df_baseline, df_optimized)

In [ ]:
df_optimized[100 * (1 - ((df_optimized.runtime_ss_after + df_optimized.runtime_de_after) / df_optimized.runtime_before)) <= -100]

In [ ]:
def combine_sort():
    data = []
    for filename in os.listdir(DATA_DIR):
        if filename.startswith('sort'):
            test, run = filename.split("--")[1].split(".")[0].split("_")
            if test not in ['optde', 'de', 'noopt', 'opt']:
                continue
            print(f"test: {test}, run: {run}")
            with open(os.path.join(DATA_DIR, filename), 'r') as f:
                for scenename, metadata in tqdm(json.load(f).items()):
                    for i, m in enumerate(metadata):
                        for oid, det in m.items():
                            data.append({
                                '_test': test,
                                'run': run,
                                'test': test_names[test],
                                'scenename': scenename,
                                'detection': {'oid': oid, 'did': det['detection_id']}
                            })
    return data


combined_sort = combine_sort()

In [ ]:
len(combined_sort)

In [ ]:
combined_sort[0]

In [ ]:
df_combined_sort = pd.DataFrame.from_dict(combined_sort)

In [ ]:
df_grouped_combined_sort = df_combined_sort.groupby(['_test', 'run', 'test', 'scenename']).count().reset_index()

In [ ]:
df_grouped_combined_sort.groupby(['scenename', '_test', 'run']).count().reset_index()

In [ ]:
df_perf

In [ ]:
df_grouped_perf = df_perf[(df_perf.stage == 'DetectionEstimation')].groupby(['name', '_test', 'run']).count().reset_index()
df_grouped_perf

In [ ]:
def remove_head(df):
    # print(df['name'])
    return '-'.join(df['name'].split('-')[2:])
df_perf.assign(scenename=df_perf.apply(remove_head, axis=1))

In [ ]:
def remove_head(df):
    return '-'.join(df['name'].split('-')[2:])

df_det_skip = (df_grouped_combined_sort
    .set_index(['scenename', '_test', 'run'])
    .join(
        df_perf[(df_perf.stage == 'DetectionEstimation')]
            .assign(scenename=df_perf.apply(remove_head, axis=1))
            .set_index(['scenename', '_test', 'run']),
        lsuffix="_det",
        rsuffix="_perf",
        how="inner"
    )
    .reset_index()
)

In [ ]:
df_det_skip

In [ ]:
df_det_skip[df_det_skip.run == '0']

In [ ]:
(alt
    .Chart(df_det_skip[(df_det_skip.run == '0') & (df_det_skip._test == 'de')])
    .mark_point()
    .encode(
        x=alt.X('detection:Q'),
        y=alt.Y('skip:Q')
    )
) | (alt
    .Chart(df_det_skip[(df_det_skip.run == '0') & (df_det_skip._test == 'de')])
    .mark_point()
    .encode(
        x=alt.X('detection:Q'),
        y=alt.Y('runtime:Q')
    )
) | (alt
    .Chart(df_det_skip[(df_det_skip.run == '0') & (df_det_skip._test == 'de')])
    .mark_point()
    .encode(
        x=alt.X('skip:Q'),
        y=alt.Y('runtime:Q')
    )
)